In [1]:
#!pip install "git+https://github.com/pycaret/pycaret.git@master"

In [23]:
from pycaret.classification import *

In [2]:
import pandas as pd
import numpy as np

In [3]:
dataset=pd.read_csv('/content/all_data_cleaned_data_bankFraud.csv')

In [4]:
dataset.shape

(284807, 33)

In [5]:
dataset.columns

Index(['id', 'Group', 'Per1', 'Per2', 'Per3', 'Per4', 'Per5', 'Per6', 'Per7',
       'Per8', 'Per9', 'Dem1', 'Dem2', 'Dem3', 'Dem4', 'Dem5', 'Dem6', 'Dem7',
       'Dem8', 'Dem9', 'Cred1', 'Cred2', 'Cred3', 'Cred4', 'Cred5', 'Cred6',
       'Normalised_FNT', 'Target', 'data', 'lambda_wt', 'geo_score',
       'instance_scores', 'qsets_normalized_tat'],
      dtype='object')

In [6]:
dataset.head()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Cred4,Cred5,Cred6,Normalised_FNT,Target,data,lambda_wt,geo_score,instance_scores,qsets_normalized_tat
0,112751,Grp169,1.070000,0.580000,0.480000,0.766667,1.233333,1.993333,0.340000,1.010000,...,0.603333,0.686667,0.673333,-245.7500,0.0,train,-0.13,0.256,-0.06,-0.700
1,18495,Grp161,0.473333,1.206667,0.883333,1.430000,0.726667,0.626667,0.810000,0.783333,...,0.670000,0.553333,0.653333,-248.0000,0.0,train,0.66,-0.250,0.52,0.140
2,23915,Grp261,1.130000,0.143333,0.946667,0.123333,0.080000,0.836667,0.056667,0.756667,...,0.670000,0.686667,0.673333,-233.1250,0.0,train,-0.51,-0.950,1.56,-0.430
3,50806,Grp198,0.636667,1.090000,0.750000,0.940000,0.743333,0.346667,0.956667,0.633333,...,0.520000,0.846667,0.760000,-249.7775,0.0,train,0.72,0.490,0.70,-0.302
4,184244,Grp228,0.560000,1.013333,0.593333,0.416667,0.773333,0.460000,0.853333,0.796667,...,0.716667,0.706667,0.673333,-247.5775,0.0,train,0.60,0.850,-0.47,-0.630


In [8]:
#for training set
training=dataset[dataset['data']=='train']

In [9]:
#for test set(rename col for test after frac =0.8)
testing=dataset[dataset['data']=='test']

In [10]:
training.shape

(227845, 33)

In [11]:
testing.shape

(56962, 33)

In [13]:
classification_model=setup(data=training,target='Target',session_id=101)

,Description,Value
0,Session id,101
1,Target,Target
2,Target type,Binary
3,Original data shape,"(227845, 33)"
4,Transformed data shape,"(227845, 33)"
5,Transformed train set shape,"(159491, 33)"
6,Transformed test set shape,"(68354, 33)"
7,Numeric features,30
8,Categorical features,2
9,Preprocess,True


In [25]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9992,0.9711,0.6302,0.8767,0.7294,0.7290,0.7409,10.5530
rf,Random Forest Classifier,0.9989,0.9423,0.4488,0.8256,0.5769,0.5764,0.6056,31.7020
ridge,Ridge Classifier,0.9987,0.9756,0.2897,0.8184,0.4250,0.4245,0.4842,0.7350
et,Extra Trees Classifier,0.9987,0.9387,0.2963,0.7890,0.4276,0.4271,0.4808,8.0680
nb,Naive Bayes,0.9986,0.9703,0.4709,0.6666,0.5463,0.5456,0.5567,0.8170
lda,Linear Discriminant Analysis,0.9985,0.8994,0.3041,0.6325,0.4089,0.4082,0.4368,1.1190
knn,K Neighbors Classifier,0.9984,0.5747,0.0583,0.6000,0.1049,0.1047,0.1835,22.3860
ada,Ada Boost Classifier,0.9984,0.9658,0.1988,0.6011,0.2917,0.2911,0.3388,12.9120
gbc,Gradient Boosting Classifier,0.9984,0.4744,0.2058,0.6209,0.2841,0.2836,0.3361,63.9480
xgboost,Extreme Gradient Boosting,0.9983,0.9064,0.1914,0.4990,0.2496,0.2491,0.2908,2.7250


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=101, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

- in fraud detection, the primary goal is to catch as many fraudulent transactions as possible. accuracy is less important because fraud cases are very rare, and missing a fraud (false negative) can be very costly. hence, high recall is critical.

- precision is secondary; some false alarms can be tolerated since they can be reviewed manually.

- i will use qda because my goal is to detect as many frauds as possible. even if it produces false positives, it does not harm the business—these can be internally reviewed or explained to customers based on compliance rules. accuracy is not the main priority here.

In [27]:
qd=create_model('qda')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9810,0.9790,0.8929,0.0769,0.1416,0.1389,0.2590
1,0.9777,0.9602,0.8889,0.0637,0.1188,0.1160,0.2345
2,0.9813,0.9544,0.8519,0.0726,0.1337,0.1310,0.2455
3,0.9799,0.9941,0.9259,0.0727,0.1348,0.1320,0.2564
4,0.9802,0.9763,0.8519,0.0689,0.1274,0.1247,0.2390
5,0.9820,0.9414,0.8571,0.0782,0.1433,0.1405,0.2557
6,0.9801,0.9718,0.9286,0.0760,0.1405,0.1377,0.2626
7,0.9829,0.9858,0.8929,0.0847,0.1548,0.1521,0.2721
8,0.9799,0.9480,0.8571,0.0704,0.1301,0.1272,0.2423


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [29]:
lr=create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9988,0.9778,0.5357,0.7143,0.6122,0.6117,0.6180
1,0.9991,0.9962,0.5185,0.8750,0.6512,0.6507,0.6732
2,0.9990,0.9878,0.5926,0.7619,0.6667,0.6662,0.6715
3,0.9994,0.9961,0.6667,0.9474,0.7826,0.7823,0.7944
4,0.9992,0.9459,0.6667,0.8182,0.7347,0.7343,0.7382
5,0.9993,0.9720,0.6429,0.9474,0.7660,0.7656,0.7801
6,0.9994,0.9720,0.7857,0.8462,0.8148,0.8145,0.8151
7,0.9995,0.9773,0.7143,1.0000,0.8333,0.8331,0.8449
8,0.9992,0.9403,0.5357,1.0000,0.6977,0.6973,0.7316


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

**From the above two model its confirmed qda is best choice:**
- for fraud detection, recall is the priority
- qda has much higher recall and consistently catches more fraud cases
- accuracy and precision are secondary here; false positives can be manually verified

# **# QDA**

In [30]:
#hyper parameter tuning

tuned_qd=tune_model(qd)
tuned_qd

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9919,0.9783,0.8571,0.1611,0.2712,0.2690,0.3696
1,0.9924,0.9793,0.8889,0.1690,0.2840,0.2820,0.3858
2,0.9924,0.9879,0.7778,0.1533,0.2561,0.2540,0.3432
3,0.9954,0.9959,0.9259,0.2604,0.4065,0.4049,0.4897
4,0.9922,0.9334,0.8148,0.1560,0.2619,0.2598,0.3546
5,0.9962,0.9771,0.8571,0.2963,0.4404,0.4389,0.5027
6,0.9933,0.9898,0.9286,0.1985,0.3270,0.3251,0.4276
7,0.9929,0.9918,0.8929,0.1838,0.3049,0.3028,0.4033
8,0.9962,0.9506,0.8571,0.2963,0.4404,0.4389,0.5027


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


QuadraticDiscriminantAnalysis(priors=None, reg_param=0.77,
                              store_covariance=False, tol=0.0001)

In [32]:
# tuned qda improves overall performance compared to prev/og qda
# recall remains high (0.855), ensuring most fraudulent transactions are detected
# precision and f1 are improved, reducing false positives and making the model more practical
# accuracy is slightly higher (0.9934), but for fraud detection, recall is still the main priority
# low standard deviation shows consistent performance across folds
# overall, tuned qda is more reliable and suitable for this fraud detection

In [33]:
evaluate_model(tuned_qd)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [37]:
testing=testing.drop(['Target'],axis=1)

In [38]:
testing.isnull().sum().sum()

0

In [40]:
# predict unseen data

unseen_prediction=predict_model(tuned_qd,data=testing)
unseen_prediction.head()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Cred5,Cred6,Normalised_FNT,data,lambda_wt,geo_score,instance_scores,qsets_normalized_tat,prediction_label,prediction_score
227845,146574,Grp229,-0.300000,1.540000,0.220000,-0.280000,0.570000,0.260000,0.700000,1.076667,...,0.823333,0.783333,-249.750000,test,0.76,0.25,-0.04,-0.426,0,1.0
227846,268759,Grp141,0.633333,0.953333,0.810000,0.466667,0.910000,0.253333,1.040000,0.550000,...,0.673333,0.673333,-249.812500,test,0.18,0.43,-0.77,-0.620,0,1.0
227847,59727,Grp188,1.043333,0.740000,0.860000,1.006667,0.583333,0.616667,0.630000,0.686667,...,0.680000,0.670000,-248.119995,test,0.39,1.32,0.11,-0.406,0,1.0
227848,151544,Grp426,1.283333,0.300000,0.576667,0.636667,0.256667,0.543333,0.356667,0.663333,...,0.676667,0.660000,-222.987503,test,1.80,-2.11,0.33,0.374,0,1.0
227849,155008,Grp443,1.186667,0.326667,0.476667,0.866667,0.436667,0.680000,0.476667,0.686667,...,0.650000,0.663333,-196.220001,test,1.89,-2.11,-0.37,-0.130,0,1.0


In [41]:
# saving the model

save_model(tuned_qd,"Bank_ATM_fraud_pycaret_colabs_")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['id', 'Per1', 'Per2', 'Per3',
                                              'Per4', 'Per5', 'Per6', 'Per7',
                                              'Per8', 'Per9', 'Dem1', 'Dem2',
                                              'Dem3', 'Dem4', 'Dem5', 'Dem6',
                                              'Dem7', 'Dem8', 'Dem9', 'Cred1',
                                              'Cred2', 'Cred3', 'Cred4', 'Cred5',
                                              'Cred6', 'Normalised_FNT',
                                              'lambda_wt', 'geo_score',
                                              'instance_scores',
                                              'qset...
                  TransformerWrapper(exclude=None, include=['Group'],
                                     transformer=TargetEncoder(c